### [Automate your stock screening using Python](https://levelup.gitconnected.com/automate-your-stock-screening-using-python-9107dda724c3)

1. 定義交易策略 <br>
2. 進行篩選 <br>
3. 自動化流程

#### 第 1 部分：定義交易策略
簡單的EMA（指數移動平均線）反彈策略。顧名思義，我們試圖在 EMA 趨勢線上找到反彈，例如 50 天 EMA。

1. 總體趨勢是上升趨勢：18 天 EMA > 50 天 EMA > 100 天 EMA
2. 隨機震盪指標 (5 , 3, 3) 低於 30，表明股票超賣。
3. 前一根蠟燭（蠟燭 2）的低點低於 EMA 50；收盤價高於 EMA 50；最後一根蠟燭（蠟燭 1）的低點高於 EMA 50。

#### 第 2 部分：執行篩選

In [1]:
import yfinance as yf 
data = yf.download("AAPL", start="2020-01-01") 
print(data)

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2019-12-31   72.482498   73.419998   72.379997   73.412498   72.039879   
2020-01-02   74.059998   75.150002   73.797501   75.087502   73.683556   
2020-01-03   74.287498   75.144997   74.125000   74.357498   72.967209   
2020-01-06   73.447502   74.989998   73.187500   74.949997   73.548615   
2020-01-07   74.959999   75.224998   74.370003   74.597504   73.202728   
...                ...         ...         ...         ...         ...   
2022-08-29  161.149994  162.899994  159.820007  161.380005  161.380005   
2022-08-30  162.130005  162.559998  157.720001  158.910004  158.910004   
2022-08-31  160.309998  160.580002  157.139999  157.220001  157.220001   
2022-09-01  156.639999  158.419998  154.669998  157.960007  157.960007   
2022-09-02  159.750000  160.360001  154.970

In [2]:
import requests
from bs4 import BeautifulSoup

def get_stock_list():
  # this is the website we're going to scrape from
  url = "https://www.malaysiastock.biz/Stock-Screener.aspx"
  response = requests.get(url, headers={'User-Agent':'test'})
  soup = BeautifulSoup(response.content, "html.parser")
  table = soup.find(id = "MainContent2_tbAllStock")
  # return the result (only ticker code) in a list
  stock_list = table.find_all('a')
  return [stock_code.get('href')[-4:] for stock_code in stock_list]

In [3]:
stock_list = get_stock_list()
print(len(stock_list))

1004


#### 2.3 創建技術指標
18 天 EMA、50 天 EMA、100 天 EMA 和隨機震盪指標(5, 3, 3)

In [5]:
def get_stock_price(ticker):
  data = yf.download(ticker, start="2021-01-01")
  return data

def add_EMA(price, day):
  return price.ewm(span=day).mean()

def add_STOCH(close, low, high, period, k, d=0): 
    STOCH_K = ((close - low.rolling(window=period).min()) / (high.rolling(window=period).max() - low.rolling(window=period).min())) * 100
    STOCH_K = STOCH_K.rolling(window=k).mean()
    if d == 0:
      return STOCH_K
    else:
      STOCH_D = STOCH_K.rolling(window=d).mean()
      return STOCH_D

stock_code = "SPY"
price_chart_df = get_stock_price(stock_code ) # + ".KL"

open = price_chart_df['Open']
close = price_chart_df['Close']
high = price_chart_df['High']
low = price_chart_df['Low']
price_chart_df['EMA18'] = add_EMA(close,18)
price_chart_df['EMA50'] = add_EMA(close,50)
price_chart_df['EMA100'] = add_EMA(close,100)
price_chart_df['STOCH_%K(5,3,3)'] = add_STOCH(close, low, high, 5, 3)
price_chart_df['STOCH_%D(5,3,3)'] = add_STOCH(close, low, high, 5, 3, 3)

[*********************100%***********************]  1 of 1 completed


In [6]:
# function to check for EMA Bounce
def check_bounce_EMA(df):
  candle1 = df.iloc[-1]
  candle2 = df.iloc[-2]
  cond1 = candle1['EMA18'] > candle1['EMA50'] > candle1['EMA100']
  cond2 = candle1['STOCH_%K(5,3,3)'] <= 30 or candle1['STOCH_%D(5,3,3)'] <= 30
  cond3 = candle2['Low'] < candle2['EMA50'] and \
          candle2['Close'] > candle2['EMA50'] and \
          candle1['Low'] > candle1 ['EMA50']
  return cond1 and cond2 and cond3

# price_chart_df is from Section 2.3
print(check_bounce_EMA(price_chart_df))

False


#### 第 3 部分：自動化流程

In [ ]:
import email
# you can add this part of code at the end of part 2
# remember: screened_list contains the result of the screening
# configure email and message
msg = email.message_from_string(", ".join(screened_list))
msg['From'] = 'benzyeh@gmail.com'
msg['To'] = 'benzyeh@gmail.com'
msg['Subject'] = "EMA Bounce Result for Today!"

s = smtplib.SMTP("smtp.gmail.com",587)
## for yahoo mail user: s = smtplib.SMTP("smtp.mail.yahoo.com",587) 
## for hotmail user: s = smtplib.SMTP("smtp.live.com",587)
s.ehlo() 
s.starttls()
s.ehlo()
s.login(email_from,"YOUR_PASSWORD")
s.sendmail(email_from, [email_to] + [email_cc], msg.as_string())
s.quit()